# **DATA** **EXTRACTION**

In [27]:
#collected from kaggle datasets
import pandas as pd
first_source_df = pd.read_csv('sample_data/Uber Drives.csv')
first_source_df.head()


,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,MILES*,PURPOSE*
0,1/1/2016 21:11,1/1/2016 21:17,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain
1,1/2/2016 1:25,1/2/2016 1:37,Business,Fort Pierce,Fort Pierce,5.0,NaN
2,1/2/2016 20:25,1/2/2016 20:38,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies
3,1/5/2016 17:31,1/5/2016 17:45,Business,Fort Pierce,Fort Pierce,4.7,Meeting
4,1/6/2016 14:42,1/6/2016 15:49,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit


In [28]:
#collected from NYC website
second_source_df= pd.read_csv('sample_data/NYC_tripdata.csv',nrows=10000)
second_source_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-08-12 16:31:24.000000,2024-08-12 17:11:20.000000,2,13.72,1,N,132,223,2,56.2,2.5,0.5,0.00,0.0,1.0,61.95,0.0,1.75
1,2,2024-08-12 16:55:27.000000,2024-08-12 17:09:13.000000,1,1.21,1,N,230,233,2,12.8,2.5,0.5,0.00,0.0,1.0,19.30,2.5,0.00
2,2,2024-08-12 16:46:29.000000,2024-08-12 16:54:03.000000,1,0.00,1,N,186,234,1,7.9,2.5,0.5,0.00,0.0,1.0,14.40,2.5,0.00
3,2,2024-08-12 16:34:43.000000,2024-08-12 16:58:32.000000,1,2.22,1,N,236,164,1,20.5,2.5,0.5,5.40,0.0,1.0,32.40,2.5,0.00
4,2,2024-08-12 16:26:00.000000,2024-08-12 16:37:08.000000,1,3.20,1,N,70,260,1,16.3,7.5,0.5,5.06,0.0,1.0,32.11,0.0,1.75


In [29]:
#scraped from consumer affairs website
third_source_df= pd.read_csv('sample_data/Uber Review Data.csv')
third_source_df.head()

,Date,Stars,Comment
0,10/29/2019,1,I had an accident with an Uber driver in Mexic...
1,10/28/2019,1,I have had my account completely hacked to whe...
2,10/27/2019,1,I requested an 8 mile ride in Boston on a Satu...
3,10/27/2019,1,I've been driving off and on with the company ...
4,10/25/2019,1,Uber is overcharging for Toll fees. When In Fl...


# **DATA TRANSFORMATION**

In [30]:


# Remove asterisks (*) from column names
first_source_df.columns = first_source_df.columns.str.replace('*', '', regex=False)


# Step 1: Select Necessary Columns
# Keep only necessary columns in each dataset, adding additional fields from the second source
first_source_df = first_source_df[['START_DATE', 'END_DATE', 'CATEGORY', 'START', 'STOP', 'MILES', 'PURPOSE']]
second_source_df = second_source_df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'tip_amount', 'total_amount', 'passenger_count']]
third_source_df = third_source_df[['Stars', 'Comment']]
# Step 2: Rename Columns for Consistency
second_source_df.rename(columns={
    'tpep_pickup_datetime': 'START_DATE',
    'tpep_dropoff_datetime': 'END_DATE',
    'trip_distance': 'MILES',
    'PULocationID': 'START',
    'DOLocationID': 'STOP',
    'fare_amount': 'fare_amount',
    'tip_amount': 'tip_amount',
    'total_amount': 'total_amount',
    'passenger_count': 'passenger_count',
    'payment_Type': 'payment_type',
}, inplace=True)

third_source_df.rename(columns={
    'Stars': 'Rating',
    'Comment': 'Comment',
}, inplace=True)


# Step 3: Transform Data Formats

# Convert date columns to datetime format
first_source_df['START_DATE'] = pd.to_datetime(first_source_df['START_DATE'], format="%m/%d/%Y %H:%M", errors='coerce')
first_source_df['END_DATE'] = pd.to_datetime(first_source_df['END_DATE'], format="%m/%d/%Y %H:%M", errors='coerce')
second_source_df['START_DATE'] = pd.to_datetime(second_source_df['START_DATE'], errors='coerce')
second_source_df['END_DATE'] = pd.to_datetime(second_source_df['END_DATE'], errors='coerce')



# Ensure MILES is numeric in both datasets
first_source_df['MILES'] = pd.to_numeric(first_source_df['MILES'], errors='coerce')
second_source_df['MILES'] = pd.to_numeric(second_source_df['MILES'], errors='coerce')


# Step 4: Handle Duplicates and Missing Values

# Remove duplicate records in  datasets
first_source_df.drop_duplicates(inplace=True)
second_source_df.drop_duplicates(inplace=True)


# Drop rows with missing values in critical fields (you can adjust this based on your needs)
first_source_df.dropna(subset=['START_DATE', 'END_DATE', 'START', 'STOP', 'MILES'], inplace=True)
second_source_df.dropna(subset=['START_DATE', 'END_DATE', 'START', 'STOP', 'MILES'], inplace=True)
third_source_df.dropna(subset=['Rating', 'Comment'], inplace=True)

# Fill missing values in PURPOSE with 'No Purpose'
first_source_df['PURPOSE'] = first_source_df['PURPOSE'].fillna('No Purpose')

# Optional: Standardize categorical fields for consistency
first_source_df['CATEGORY'] = first_source_df['CATEGORY'].str.lower()
first_source_df['PURPOSE'] = first_source_df['PURPOSE'].str.lower()



# Step 5: Combine Datasets

first_source_df[['payment_type', 'fare_amount', 'tip_amount', 'total_amount', 'passenger_count']] = second_source_df[['payment_type', 'fare_amount', 'tip_amount', 'total_amount', 'passenger_count']]
first_source_df[['Rating','Comment']] = third_source_df[['Rating','Comment']]

first_source_df['TIME'] = first_source_df['START_DATE'].dt.time
first_source_df['START_DATE'] = first_source_df['START_DATE'].dt.date
first_source_df['END_DATE'] = first_source_df['END_DATE'].dt.date

# Step 6: Enrich Data

# a function to categorize the time of day
def categorize_time(time):
    hour = time.hour
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

first_source_df['TIME'] = first_source_df['TIME'].apply(categorize_time)
first_source_df['TIME'] = first_source_df['TIME'].dropna()

combined_df = first_source_df

# Preview the final combined dataset with the new fie
# combined_df.to_csv('sample_data/combined_data.csv', index=False)
combined_df.head(100)

,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE,payment_type,fare_amount,tip_amount,total_amount,passenger_count,Rating,Comment,TIME
0,2016-01-01,2016-01-01,business,Fort Pierce,Fort Pierce,5.1,meal/entertain,2,56.2,0.00,61.95,2,1,I had an accident with an Uber driver in Mexic...,Night
1,2016-01-02,2016-01-02,business,Fort Pierce,Fort Pierce,5.0,no purpose,2,12.8,0.00,19.30,1,1,I have had my account completely hacked to whe...,Night
2,2016-01-02,2016-01-02,business,Fort Pierce,Fort Pierce,4.8,errand/supplies,1,7.9,0.00,14.40,1,1,I requested an 8 mile ride in Boston on a Satu...,Evening
3,2016-01-05,2016-01-05,business,Fort Pierce,Fort Pierce,4.7,meeting,1,20.5,5.40,32.40,1,1,I've been driving off and on with the company ...,Evening
4,2016-01-06,2016-01-06,business,Fort Pierce,West Palm Beach,63.7,customer visit,1,16.3,5.06,32.11,1,1,Uber is overcharging for Toll fees. When In Fl...,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2016-02-12,2016-02-12,business,Cary,Durham,8.5,temporary site,1,11.4,3.58,21.48,1,1,"I moved back to Huntsville, Alabama in May of ...",Morning
96,2016-02-12,2016-02-12,business,Durham,Morrisville,2.6,temporary site,1,19.1,4.62,29.47,2,1,Scheduled a ride to the Tampa Airport. Driver ...,Morning
97,2016-02-12,2016-02-12,business,Morrisville,Raleigh,17.0,customer visit,1,9.3,3.95,19.75,1,3,I use Uber when I miss the bus after work some...,Morning
98,2016-02-12,2016-02-12,business,Raleigh,Cary,18.0,meeting,1,14.2,3.00,23.70,1,4,"Overall a very marvellous travelling facility,...",Afternoon


# LOADING DATA INTO POSTGRESQL DATABASE

In [32]:
import pandas as pd
from sqlalchemy import create_engine

# Sample connection string; replace with your actual PostgreSQL credentials
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'localhost'
USER = 'postgres'
PASSWORD = 'pakistan'
PORT = 5432                          # Default PostgreSQL port is 5432
DATABASE = 'UBER DATA WAREHOUSE'

# Create a connection string and engine
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

# Load the DataFrame into PostgreSQL
table_name = 'UBER_DATA'  # Choose your table name
combined_df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"Data successfully loaded into table '{table_name}' in the '{DATABASE}' database.")


Data successfully loaded into table 'UBER_DATA' in the 'UBER DATA WAREHOUSE' database.
